In [1]:
import os
import sys
from shutil import copy, rmtree
from tqdm import tqdm

PATH_TO_ENUMLIB = '/Users/dmitry.volkov/enumlib'

In [2]:
def create_job_script(path, job_id='JobName'):
    """
    material_id(str) - preferable name of your job in squeue,
    and also the name of the folder for your vasp files.
    as a result in the folder with name 'material_id' will be created job scrypt
    """
    job_script_text = f"""#!/bin/bash
#SBATCH -t 01:00:00
#SBATCH -N 1
#SBATCH -n 4
#SBATCH -J {job_id}
#SBATCH --output=log
#SBATCH --error=err

module load mpi/impi-5.0.3 intel/mkl-11.2.3 vasp/vasp-5.4.4
mpirun vasp_std"""
    with open(f'{path}/jobscript', 'w') as job:
        job.writelines(job_script_text)

In [3]:
def get_number_of_structures(enum_out='struct_enum.out'):
    """
    Read file 'struct_enum.out' and
    returns number of generated supercells after
    the work of enum.x
    """
    with open("struct_enum.out", "r") as file:
        lastline = (list(file)[-1])
        num_of_structures = int(lastline[:11])
#         print(num_of_structures)
    return num_of_structures

In [4]:
def get_vasp_structures(path_to_enum=PATH_TO_ENUMLIB):
    """
    This function read 'struct_enum.out'
    and generate POSCAR type files for all produced supercells.
    """
    number_of_structures = get_number_of_structures() #int number of generated supercells
    makeStrPath = os.path.join(PATH_TO_ENUMLIB, 'aux_src/makeStr.py')
    os.system(f'python {makeStrPath} 1 {number_of_structures}')
    print(f'Generated {number_of_structures} supercells')

In [5]:
def vasp_inputs_creator(out_dir='vasp_inputs', input_directory='user_inputs'):
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)
    current_path = os.getcwd()
    num_of_structures = get_number_of_structures()
    for i in tqdm(range(num_of_structures)):
        tmp_path = os.path.join(current_path, out_dir, f'afm{i + 1}')
        os.makedirs(tmp_path, exist_ok=True)
        create_job_script(tmp_path, job_id=f'afm{i + 1}')
        poscar_prettifier(in_path=f'vasp.{i + 1}', out_path=os.path.join(tmp_path, 'POSCAR'))
        copy(os.path.join(input_directory, 'INCAR'), os.path.join(tmp_path, 'INCAR'))
        copy(os.path.join(input_directory, 'POTCAR'), os.path.join(tmp_path, 'POTCAR'))
        copy(os.path.join(input_directory, 'KPOINTS'), os.path.join(tmp_path, 'KPOINTS'))
        print(f'Created: {tmp_path}')

In [6]:
def poscar_prettifier(in_path, out_path, atom_types='Eu O', atom_ratio='2 2'):
    """
    Make the POSCAR readable for the SIMAN software
    Keeping it fully functional for VASP calculations.
    
    Parameters:
        in_path(str) - path for the POSCAR to prettifiy
        out_path(str) - preferable path & filename for output pretiffyed POSCAR
                        is path not exists it will be created
        atom_types(str) - type of atoms you wanna see on a 6th row
        atom_ratio(str) - preferable ratio of specified atoms
    """
    out_dir, out_file_name = os.path.split(out_path)
    
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)
        
    out = open(f"{out_path}", "wt")
    
    with open(f'{in_path}', 'rt') as f:
        in_text = f.readlines()
    in_text[5] = f'{atom_types}\n'
    in_text[6] = f'{atom_ratio}\ndirect\n'

    for st in in_text:
        if st.startswith('  '):
            st = st.replace('  ', '', 1)
        elif st.startswith(' '):
            st = st.replace(' ', '', 1)
        out.write(st)
    out.close()
    print(f'{out_path} is prettified')

In [8]:
def clean_all():
    vasp_usless = [f'vasp.{i}' for i in range(1, get_number_of_structures() + 1)]
    usless_files = ['debug_conc_check.out',
                    'debug_dvec_rots.out',
                    'debug_get_rotation_perms_lists.out',
                    'debug_site_restrictions.out',
                    'readcheck_enum.out',
                    'symops_enum_parent_lattice.out',
                    'VERSION.enum',
                   'struct_enum.out'
                   ] + vasp_usless
    for file in tqdm(usless_files):
        try:
            os.remove(file)
        except:
            print(f'File {file} already removed!')
    rmtree('./vasp_inputs/', ignore_errors=True)
    rmtree('./siman_rdy/', ignore_errors=True)

# Main

In [9]:
enum_exe_path = os.path.join(PATH_TO_ENUMLIB, 'src/enum.x')

os.system(enum_exe_path)

current_path = os.getcwd()
get_vasp_structures(path_to_enum=PATH_TO_ENUMLIB)
vasp_inputs_creator(out_dir='vasp_inputs', input_directory='user_inputs')

In [13]:
for i in range(1, 8):
    poscar_prettifier(f'vasp.{i}',
                      f'siman_rdy/POSCAR_{i}',
                      atom_types='Eu Sm O',
                      atom_ratio='1 1 2')
    
for i in range(8, get_number_of_structures() + 1):
    poscar_prettifier(f'vasp.{i}',
                      f'siman_rdy/POSCAR_{i}',
                      atom_types='Eu Sm O',
                      atom_ratio='2 2 4')

siman_rdy/POSCAR_1 is prettified
siman_rdy/POSCAR_2 is prettified
siman_rdy/POSCAR_3 is prettified
siman_rdy/POSCAR_4 is prettified
siman_rdy/POSCAR_5 is prettified
siman_rdy/POSCAR_6 is prettified
siman_rdy/POSCAR_7 is prettified
siman_rdy/POSCAR_8 is prettified
siman_rdy/POSCAR_9 is prettified
siman_rdy/POSCAR_10 is prettified
siman_rdy/POSCAR_11 is prettified
siman_rdy/POSCAR_12 is prettified
siman_rdy/POSCAR_13 is prettified
siman_rdy/POSCAR_14 is prettified
siman_rdy/POSCAR_15 is prettified
siman_rdy/POSCAR_16 is prettified
siman_rdy/POSCAR_17 is prettified
siman_rdy/POSCAR_18 is prettified
siman_rdy/POSCAR_19 is prettified
siman_rdy/POSCAR_20 is prettified
siman_rdy/POSCAR_21 is prettified
siman_rdy/POSCAR_22 is prettified
siman_rdy/POSCAR_23 is prettified
siman_rdy/POSCAR_24 is prettified
siman_rdy/POSCAR_25 is prettified
siman_rdy/POSCAR_26 is prettified
siman_rdy/POSCAR_27 is prettified
siman_rdy/POSCAR_28 is prettified
siman_rdy/POSCAR_29 is prettified
siman_rdy/POSCAR_30 is 

In [10]:
#  clean_all() 